# Transport Example with Pyomo
* [Pyomo Documentation](https://pyomo.readthedocs.io/en/latest/)
* [Gallery](https://github.com/Pyomo/PyomoGallery/wiki)
* [Accessing Pyomo variable values and objective function value](http://hselab.org/pyomo-get-variable-values.html)

In [29]:
from pyomo.environ import *
from IPython.display import display, Math, Latex # latex equations
from IPython.display import Image # images

<img src="transport.png" width="400">

__Variables__:
* $x_{ij}$: quantity of product from factory $i$ to market $j$
* $c_{ij}$: cost of send one unit of product from factory $i$ to market $j$
* $z$:  total cost of shipment

__Objective function__:
* (1) $Min: z =  \sum_{i \in \Omega_f} \sum_{j \in \Omega_m} x_{ij}  c_{ij}   $

__Constraints__:
* Demand supply:
(2) $\sum_{i \in \Omega_f} x_{ij} \geq d_j; \forall  j \in \Omega_m$


* Production limits 
(3) $ \sum_{j \in \Omega_m} x_{ij} \leq p_i; \forall  i \in \Omega_f$

#### Abstract model
(no data in the code, only the equations)

In [30]:
model = AbstractModel()

#### Define the sets

Markets: $\Omega_m = \{m_1, m_2, \ldots, m_s\}$

Factories: $\Omega_f = \{f_1, f_2, \ldots, f_r\}$

In [31]:
model.markets = Set()
model.factories = Set()

#### Define the variables

production: $p_{i}= (p_1, p_2, \ldots, p_r)$

demand: $d_{i}= (d_1, d_2, \ldots, d_s)$

In [32]:
model.production = Param(model.factories)
model.demand = Param(model.markets)

#### Define the matrix $M(f,m)$ factories to markets costs
$$
M(f,m) = \begin{bmatrix}
    x_{11}       & \dots & \dots  & x_{1s} \\
    x_{21}       & \dots & \dots  & x_{2s} \\
    \vdots       & \vdots & \vdots & \vdots \\
    x_{r1}       & \dots & \dots & x_{rs}
\end{bmatrix}
$$

In [33]:
model.costs = Param(model.factories,model.markets)

#### Define the variable of units of product from factory $i$ to market $j$ $x_{i,j}$

In [34]:
model.units = Var(model.factories, model.markets, within = NonNegativeReals)

#### Define objective function (1)

In [35]:
def Totalcost(model):
    return sum(model.costs[n,i]*model.units[n,i]
               for n in model.factories
               for i in model.markets)

#### Define objective 

In [36]:
model.finalcost = Objective(rule = Totalcost)

#### Define constraint function (2)

In [37]:
def MinDemand(model,markets):
    return sum(model.units[i,markets] for i in model.factories) >= model.demand[markets]

#### Define constraint function (3)

In [38]:
def MaxProduction(model,factories):
    return sum(model.units[factories,j] for j in model.markets) <= model.production[factories]

#### Define constraints 

In [39]:
model.DemandConstraint = Constraint(model.markets, rule = MinDemand)
model.ProductionConstraint = Constraint(model.factories, rule = MaxProduction)

#### Now create function to build model  

In [74]:
def create_model():
    """ This function create an abstract model for the transport problem
    
    :return model:
    
    """

    from pyomo.environ import *
    
    model = AbstractModel()
    # Define the sets
    model.markets = Set()
    model.factories = Set()

    # Define the variables
    model.production = Param(model.factories)
    model.demand = Param(model.markets, mutable=True) # mutable to change the values with instance its created

    # Define the matrix of cost of send one profuct from factories to markets
    model.costs = Param(model.factories,model.markets)

    # Define the variable of units of product from factory $i$ to market $j$ $x_{i,j}$
    model.units = Var(model.factories, model.markets, bounds=(0,1000))

    # Define objective function (1)
    def Totalcost(model):
        return sum(model.costs[n,i]*model.units[n,i]
                   for n in model.factories
                   for i in model.markets)

    # Define objective
    model.finalcost = Objective(rule = Totalcost)

    # Define constraint function
    def MinDemand(model,markets):
        return sum(model.units[i,markets] for i in model.factories) >= model.demand[markets]

    # Define constraint function
    def MaxProduction(model,factories):
        return sum(model.units[factories,j] for j in model.markets) <= model.production[factories]

    # Define constraints
    model.DemandConstraint = Constraint(model.markets, rule = MinDemand)
    model.ProductionConstraint = Constraint(model.factories, rule = MaxProduction)

    return model

model = create_model()

#### instantiate the model with the .dat file and solve it

In [ ]:
# %load transport_data_v1.dat

In [59]:
instance = model.create_instance('transport_data_v1.dat')

#### modify the demand parameter if you want 

In [ ]:
#instance.demand['Alicante'].value = 100
#instance.demand['Castellon'] = 80

#### choose solver and set options

In [62]:
optimizer = SolverFactory('glpk')
#optimizer.options['max_iter'] = 100000

### solve problem

In [64]:
results = optimizer.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmpztfnsybb.glpk.raw
 --wglp /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmp5nmc07xb.glpk.glp
 --cpxlp /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmp8ldguux6.pyomo.lp
Reading problem data from '/var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmp8ldguux6.pyomo.lp'...
10 rows, 21 columns, 41 non-zeros
119 lines were read
Writing problem data to '/var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmp5nmc07xb.glpk.glp'...
125 lines were written
GLPK Simplex Optimizer, v4.65
10 rows, 21 columns, 41 non-zeros
Preprocessing...
9 rows, 20 columns, 40 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 9
      0: obj =   0.000000000e+00 inf =   7.250e+02 (5)
      6: obj =   4.281750000e+03 inf =   0.000e+00 (0)
*    11: o

#### examine results

In [78]:
print(results)
results.write() # write results in yml file


Problem: 
- Name: unknown
  Lower bound: 2328.75
  Upper bound: 2328.75
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 21
  Number of nonzeros: 41
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04378771781921387
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2328.75
  Upper bound: 2328.75
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 21
  Number of nonzeros: 41
  Sense: mini

In [68]:
print('Results')
for f in instance.factories:
    for m in instance.markets:
        if instance.units[(f, m)]!=0:
            print('From factory',f,'to market',m, 'x:',instance.units[(f, m)].value)

Results
From factory Madrid to market Vitoria x: 75.0
From factory Madrid to market Caceres x: 175.0
From factory Barcelona to market Castellon x: 150.0
From factory Barcelona to market Vitoria x: 25.0
From factory Almeria to market Alicante x: 150.0
From factory Almeria to market Cadiz x: 150.0


In [72]:
print('All Results')
instance.solutions.load_from(results)
for v in instance.component_objects(Var, active=True):
    print ("Variable",v)
    varobject = getattr(instance, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)

All Results
Variable units
    ('Madrid', 'Alicante') 0.0
    ('Madrid', 'Castellon') 0.0
    ('Madrid', 'Vitoria') 75.0
    ('Madrid', 'Cadiz') 0.0
    ('Madrid', 'Caceres') 175.0
    ('Barcelona', 'Alicante') 0.0
    ('Barcelona', 'Castellon') 150.0
    ('Barcelona', 'Vitoria') 25.0
    ('Barcelona', 'Cadiz') 0.0
    ('Barcelona', 'Caceres') 0.0
    ('Almeria', 'Alicante') 150.0
    ('Almeria', 'Castellon') 0.0
    ('Almeria', 'Vitoria') 0.0
    ('Almeria', 'Cadiz') 150.0
    ('Almeria', 'Caceres') 0.0
    ('Coruna', 'Alicante') 0.0
    ('Coruna', 'Castellon') 0.0
    ('Coruna', 'Vitoria') 0.0
    ('Coruna', 'Cadiz') 0.0
    ('Coruna', 'Caceres') 0.0


In [73]:
#instance.finalcost.expr()
print('total cost:',value(instance.finalcost),'euros')

total cost: 2328.75 euros


 #### Alternative include in a .py the model function builder and solve it using:

In [52]:
#!pyomo solve transport_example_pyomo.py transport_data.dat --solver=glpk --summary

#### or just write all code below in a .py file and execute it

In [ ]:
#!python transport_example_pyomo.py